Julio B. Roll - PS5

Brock and Mirman model with SMM.

## 1) Brock and Mirman

Brock and Mirman Model:

$$ (c_t)^{-1} - \beta E[r_{t+1}(c_{t+1})^{-1}] = 0 \tag{1} $$ <br>
$$ c_t + k_{t+1} - \omega_t - r_tk_t = 0 \tag{2} $$ <br>
$$ \omega_t - (1 - \alpha)e^{z_t}(k_t)^\alpha = 0 \tag{3} $$ <br>
$$ r_t - \alpha e^{z_t}(k_t)^{\alpha - 1} = 0 \tag{4} $$ <br>
$$ z_t = \rho z_{t-1} + (1 - \rho)\mu + \epsilon_t \tag{5} $$ <br>
$$ y_t = e^{z_{t}} k_t^{\alpha} \tag{6} $$ <br>
$$ \text{where}\: \epsilon_t \sim \mathrm{N}(0, \sigma^2)\: \text{and}\: \alpha,\beta \in (0,1), \: \mu, \sigma > 0, \: \rho \in (-1,1) $$

Optimal savings: $$k_{t+1} = \alpha\beta e^{z_{t}} k_t^{\alpha} \tag{7}$$

In [1]:
# importing our data series
import numpy as np
import scipy.stats as sts
import matplotlib.pyplot as plt
import scipy.optimize as opt
import scipy.special as spc
import scipy.integrate as intgr
import numpy.linalg as lin
import distribs
%matplotlib notebook

macro_data = np.loadtxt('NewMacroSeries.txt', delimiter = ',')
c_t = macro_data[:,0]
k_t = macro_data[:,1]
omega_t = macro_data[:,2]
r_t = macro_data[:,3]
y_t = macro_data[:,4]

### Question a)

The idea is to simulate the BM model using SMM.

Let's define some functions to help us (NB: due to notation, $z_t$ starts at t = 0, the others at t = 1, I'm fixing this by deleting the virtual values in $z_t$ and $k_t$):

In [2]:
def generator(alpha, beta, rho, mu, sigma, eps_t, T, S):
    '''
    --------------------------------------------------------------------
    This function simulates the series for given parameters.
    --------------------------------------------------------------------
    INPUTS:
    alpha     = scalar, 0 < alpha < 1
    beta      = scalar, 0 < beta < 1
    rho       = scalar, -1 < rho < 1
    mu        = scalar > 0
    sigma     = scalar > 0        
    eps_t     = (T, S) matrix, vector of normally distributed errors
    T         = scalar > 0, # periods
    S         = scalar > 0, # simulations
    
    OTHER FUNCTIONS AND FILES CALLED BY THIS FUNCTION: None
    
    OBJECTS CREATED WITHIN FUNCTION:
    z_t       = (T, S) matrix, simulated TFP
    k_sim_t   = (T, S) matrix, simulated capital
    w_sim_t   = (T, S) matrix, simulated wages
    r_sim_t   = (T, S) matrix, simulated interest rates
    c_sim_t   = (T, S) matrix, simulated consumption
    y_sim_t   = (T, S) matrix, simulated output
    
    FILES CREATED BY THIS FUNCTION: None
    
    RETURNS: z_t, k_sim_t, w_sim_t, r_sim_t, c_sim_t, y_sim_t
    '''
    z_t = np.empty([T+1,S])
    z_t[0, :] = mu
    k_sim_t = np.empty([T+1,S])
    k_sim_t[0, :] = np.mean(k_t)
    
    for i in range(T):
        z_t[i+1,:] = rho * z_t[i,:] + (1 - rho) * mu + eps_t[i,:]
        k_sim_t[i+1,:] = alpha * beta * np.exp(z_t[i+1,:]) * k_sim_t[i,:] ** alpha

    z_t = np.delete(z_t, (0), axis=0)
    k_sim_t = np.delete(k_sim_t, (T), axis=0)
    w_sim_t = (1 - alpha) * np.exp(z_t) * k_sim_t ** alpha
    r_sim_t = alpha * np.exp(z_t) * k_sim_t ** (alpha - 1)
    c_sim_t = w_sim_t + r_sim_t * k_sim_t - k_sim_t
    y_sim_t = np.exp(z_t) * k_sim_t ** alpha
            
    return z_t, k_sim_t, w_sim_t, r_sim_t, c_sim_t, y_sim_t

Regarding our moments:

In [3]:
def vcorrcoef(x,y):
    '''
    --------------------------------------------------------------------
    This function computes the correlation coeficient between each column
    of two matrices x and y. The numpy fct doesn't have an "axis" option,
    so we have to do it manually.
    --------------------------------------------------------------------
    RETURNS: r
    '''
    xm = np.mean(x, axis=0)
    ym = np.mean(y, axis=0)
    r_num = np.sum((x - xm) * (y - ym), axis=0)
    r_den = np.sqrt(np.sum((x - xm)**2, axis=0) * np.sum((y - ym)**2, axis=0))
    r = r_num/r_den
    return r

In [4]:
def data_moments(c_t, k_t, y_t):
    '''
    --------------------------------------------------------------------
    This function computes the six data moments for SMM
    (mean(c_t), mean(k_t), mean(c_t/y_t), var(y_t), corr(c_t, c_t-1), 
    corr(c_t, k_t)) from both the actual data and from the simulated data.
    --------------------------------------------------------------------
    INPUTS:
    c_t   = (N, S) matrix, (N,) or vector, consumption, either real world or 
            simulated. Real world data will come in the form (N,).
            Simulated data comes in the form (N,) or (N, S). 
    k_t   = (N, S) matrix, (N,) or vector, capital, either real world or 
            simulated. Real world data will come in the form (N,).
            Simulated data comes in the form (N,) or (N, S). 
    y_t   = (N, S) matrix, (N,) or vector, output, either real world or 
            simulated. Real world data will come in the form (N,).
            Simulated data comes in the form (N,) or (N, S).         
    
    OTHER FUNCTIONS AND FILES CALLED BY THIS FUNCTION: None
    
    OBJECTS CREATED WITHIN FUNCTION:
    mean_ct   = scalar or (S,) vector, mean value of c_t
    mean_kt   = scalar or (S,) vector, mean value of k_t
    mean_ctyt = scalar or (S,) vector, mean value of c_t/y_t
    var_yt    = scalar > 0 or (S,) vector, variance of y_t
    corr_ctct = scalar in (-1,1) or (S,) vector, corr of c_t, c_t-1
    corr_ctkt = scalar in (-1,1) or (S,) vector, corr of c_t, k_t
    
    FILES CREATED BY THIS FUNCTION: None
    
    RETURNS: mean_ct, mean_kt, mean_ctyt, var_yt, corr_ctct, corr_ctkt
    --------------------------------------------------------------------
    '''
    if len(np.array(c_t).shape) == 1:
        mean_ct = np.mean(c_t)
        mean_kt = np.mean(k_t)
        mean_ctyt = np.mean(c_t / y_t)
        var_yt = np.var(y_t)
        corr_ctct =  np.corrcoef(np.array([c_t[0:len(c_t)-1], c_t[1:len(c_t)]]))[0,1]
        corr_ctkt = np.corrcoef(c_t, k_t)[0,1]
        
    elif len(np.array(c_t).shape) == 2:
        mean_ct = np.mean(c_t, axis = 0)
        mean_kt = np.mean(k_t, axis = 0)
        mean_ctyt = np.mean(np.array(c_t) / np.array(y_t), axis = 0)
        var_yt = np.var(y_t, axis = 0)
        corr_ctct =  vcorrcoef(c_t[0:len(c_t)-1], c_t[1:len(c_t)])
        corr_ctkt = vcorrcoef(c_t, k_t)
    
    return mean_ct, mean_kt, mean_ctyt, var_yt, corr_ctct, corr_ctkt

Let's also define our usual minimization functions:

In [20]:
def err_vec(k_t, c_t, y_t, k_sim_t, c_sim_t, y_sim_t, simple):
    '''
    --------------------------------------------------------------------
    This function computes the vector of moment errors (in percent
    deviation from the data moment vector) for SMM.
    --------------------------------------------------------------------
    INPUTS:
    c_t | c_sim_t   = (N, S) matrix, (N,) or vector, consumption, either real world or 
                      simulated. Real world data will come in the form (N,).
                      Simulated data comes in the form (N,) or (N, S). 
    k_t | k_sim_t   = (N, S) matrix, (N,) or vector, capital, either real world or 
                      simulated. Real world data will come in the form (N,).
                      Simulated data comes in the form (N,) or (N, S). 
    y_t | y_sim_t   = (N, S) matrix, (N,) or vector, output, either real world or 
                      simulated. Real world data will come in the form (N,).
                      Simulated data comes in the form (N,) or (N, S). 
    simple          = boolean, =True if errors are simple difference, =False
                      if errors are percent deviation from data moments
    
    OTHER FUNCTIONS AND FILES CALLED BY THIS FUNCTION:
        data_moments()
        
    OBJECTS CREATED WITHIN FUNCTION:
    mean_ct_data   = scalar or (S,) vector, mean value of c_t
    mean_kt_data   = scalar or (S,) vector, mean value of k_t
    mean_ctyt_data = scalar or (S,) vector, mean value of c_t/y_t
    var_yt_data    = scalar > 0 or (S,) vector, variance of y_t
    corr_ctct_data = scalar in (-1,1) or (S,) vector, corr of c_t, c_t-1
    corr_ctkt_data = scalar in (-1,1) or (S,) vector, corr of c_t, k_t
    moms_data      = (6, 1) matrix, column vector of six data moments
    mean_ct_sim    = scalar or (S,) vector, mean value of c_t
    mean_kt_sim    = scalar or (S,) vector, mean value of k_t
    mean_ctyt_sim  = scalar or (S,) vector, mean value of c_t/y_t
    var_yt_sim     = scalar > 0 or (S,) vector, variance of y_t
    corr_ctct_sim  = scalar in (-1,1) or (S,) vector, corr of c_t, c_t-1
    corr_ctkt_sim  = scalar in (-1,1) or (S,) vector, corr of c_t, k_t
    moms_model     = (6, 1) matrix, column vector of six model moments
    err_vec        = (6, 1) matrix, column vector of six moment error
                     functions
    
    FILES CREATED BY THIS FUNCTION: None
    
    RETURNS: err_vec
    --------------------------------------------------------------------
    '''
    (mean_ct_data, mean_kt_data, mean_ctyt_data, var_yt_data,
    corr_ctct_data, corr_ctkt_data) = data_moments(k_t, c_t, y_t)   
    moms_data = np.array([[mean_ct_data], [mean_kt_data], [mean_ctyt_data], [var_yt_data],
                          [corr_ctct_data], [corr_ctkt_data]])
    (mean_ct_sim, mean_kt_sim, mean_ctyt_sim, var_yt_sim,
    corr_ctct_sim, corr_ctkt_sim) = data_moments(k_sim_t, c_sim_t, y_sim_t)    
    mean_ct_mod = np.mean(mean_ct_sim)
    mean_kt_mod = np.mean(mean_kt_sim)
    mean_ctyt_mod = np.mean(mean_ctyt_sim)
    var_yt_mod = np.mean(var_yt_sim)
    corr_ctct_mod = np.mean(corr_ctct_sim)
    corr_ctkt_mod = np.mean(corr_ctkt_sim)
    moms_model = np.array([[mean_ct_mod], [mean_kt_mod], [mean_ctyt_mod], [var_yt_mod], 
                          [corr_ctct_mod], [corr_ctkt_mod]])
    if simple:
        err_vec = moms_model - moms_data
    else:
        err_vec = (moms_model - moms_data) / moms_data
    
    return err_vec


def criterion(params, *args):
    '''
    --------------------------------------------------------------------
    This function computes the SMM weighted sum of squared moment errors
    criterion function value given parameter values and an estimate of
    the weighting matrix.
    --------------------------------------------------------------------
    INPUTS:
    params    = (4,) vector, ([])
    alpha     = scalar, 0 < alpha < 1
    rho       = scalar, -1 < rho < 1
    mu        = scalar > 0
    sigma     = scalar > 0
    args      = length 8 tuple,
                (c_t, k_t, y_t, unif_vals, W_hat)
    c_t       = (N, S) matrix, (N,) or vector, consumption, either real world or 
                simulated. Real world data will come in the form (N,).
                Simulated data comes in the form (N,) or (N, S). 
    k_t       = (N, S) matrix, (N,) or vector, capital, either real world or 
                simulated. Real world data will come in the form (N,).
                Simulated data comes in the form (N,) or (N, S). 
    y_t       = (N, S) matrix, (N,) or vector, output, either real world or 
                simulated. Real world data will come in the form (N,).
                Simulated data comes in the form (N,) or (N, S). 
    beta      = scalar, 0 < beta < 1
    unif_vals = (N, S) matrix, matrix of draws from U(0,1) distribution.
                This fixes the seed of the draws for the simulations
    W_hat     = (R, R) matrix, estimate of optimal weighting matrix
    
   
    OBJECTS CREATED WITHIN FUNCTION:
    eps_t      = normally distributed errors
    err        = (6, 1) matrix, vector of six moment error functions
    crit_val   = scalar > 0, GMM criterion function value
    
    FILES CREATED BY THIS FUNCTION: None
    
    RETURNS: crit_val
    --------------------------------------------------------------------
    '''
    alpha, rho, mu, sigma = params
    c_t, k_t, y_t, beta, unif_vals, W_hat, S, T = args
    eps_t = sts.norm.ppf(unif_vals, loc=0, scale=sigma)
    z_t, k_sim_t, w_sim_t, r_sim_t, c_sim_t, y_sim_t = generator(alpha, beta, rho, mu,
                                                                 sigma, eps_t, T, S)
    (mean_ct_data, mean_kt_data, mean_ctyt_data, var_yt_data, corr_ctct_data,
    corr_ctkt_data) = data_moments(c_t, k_t, y_t)
    (mean_ct_sim, mean_kt_sim, mean_ctyt_sim, var_yt_sim, corr_ctct_sim,
    corr_ctkt_sim) = data_moments(c_sim_t, k_sim_t, y_sim_t)
    err = err_vec(c_t, k_t, y_t, c_sim_t, k_sim_t, y_sim_t, simple=False)
    crit_val = np.dot(np.dot(err.T, W_hat), err) 
    
    return crit_val

SMM routine: let's set our global and initial parameters (we'll use $W = I$):

In [21]:
S = 1000
T = 100
beta = 0.99
alpha_init = 0.5
rho_init = 0.5
mu_init = 8
sigma_init = 0.5
unif_vals = sts.uniform.rvs(0, 1, size=(T, S))

params_init = np.array([alpha_init, rho_init, mu_init, sigma_init])
W_hat = np.eye(6)
smm_args = (c_t, k_t, y_t, beta, unif_vals, W_hat, S, T)
results1 = opt.minimize(criterion, params_init, args=(smm_args),
                          method='L-BFGS-B', bounds=((0.01, 1 - 0.01), (-1 + 0.01, 1 - 0.01),
                                                    (5, 14), (0.01, 1 + 0.01)))
alpha_SMM, rho_SMM, mu_SMM, sigma_SMM = results1.x
print('alpha_SMM=', alpha_SMM, 'rho_SMM=', rho_SMM, 'mu_SMM=', mu_SMM,
      'sigma_SMM=', sigma_SMM)

alpha_SMM= 0.421003315084 rho_SMM= 0.99 mu_SMM= 9.91199426198 sigma_SMM= 0.0499435019757


In [22]:
results1

      fun: array([[ 0.00064195]])
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -7.01934171e-07,  -8.47468540e-02,  -6.26018334e-08,
        -2.18822356e-06])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 340
      nit: 40
   status: 0
  success: True
        x: array([ 0.42100332,  0.99      ,  9.91199426,  0.0499435 ])

Let's check our moments:

In [23]:
(mean_ct_data, mean_kt_data, mean_ctyt_data, var_yt_data,
    corr_ctct_data, corr_ctkt_data) = data_moments(k_t, c_t, y_t)   
print('Data mean c_t =', mean_ct_data, ', Data mean k_t =', mean_kt_data,
     ', Data mean c_t/k_t =', mean_ctyt_data, ',\nData variance y_t =', var_yt_data,
     ', Data corr c_t c_t-1 =', corr_ctct_data, ',\nData corr c_t k_t =', corr_ctkt_data)
eps_t_SMM = sts.norm.ppf(unif_vals, loc=0, scale=sigma_SMM)
z_t, k_sim_t, w_sim_t, r_sim_t, c_sim_t, y_sim_t = generator(alpha_SMM, beta, rho_SMM, mu_SMM,
                                                                 sigma_SMM, eps_t_SMM, T, S)
(mean_ct_sim, mean_kt_sim, mean_ctyt_sim, var_yt_sim, corr_ctct_sim,
corr_ctkt_sim) = data_moments(c_sim_t, k_sim_t, y_sim_t)
mean_ct_mod = np.mean(mean_ct_sim)
mean_kt_mod = np.mean(mean_kt_sim)
mean_ctyt_mod = np.mean(mean_ctyt_sim)
var_yt_mod = np.mean(var_yt_sim)
corr_ctct_mod = np.mean(corr_ctct_sim)
corr_ctkt_mod = np.mean(corr_ctkt_sim)
print('Model mean c_t =', mean_ct_mod, ', Model mean k_t =', mean_kt_mod,
     ', Model mean c_t/k_t =', mean_ctyt_mod, ',\nModel variance y_t =', var_yt_mod,
     ', Model corr c_t c_t-1 =', corr_ctct_mod, ',\nModel corr c_t k_t =', corr_ctkt_mod)
err_1 = err_vec(k_t, c_t, y_t, k_sim_t, c_sim_t, y_sim_t, False).reshape(6,)
print('Error vector =', err_1)
print('Criterion =', results1.fun)

Data mean c_t = 6643985.1383 , Data mean k_t = 9281790.48567 , Data mean c_t/k_t = 0.420745434314 ,
Data variance y_t = 2.83778250589e+13 , Data corr c_t c_t-1 = 0.941096816591 ,
Data corr c_t k_t = 0.940803053798
Model mean c_t = 9300488.52455 , Model mean k_t = 6636965.73538 , Model mean c_t/k_t = 0.582207050049 ,
Model variance y_t = 2.83740059102e+13 , Model corr c_t c_t-1 = 0.93271224467 ,
Model corr c_t k_t = 0.918628816083
Error vector = [-0.0010565   0.00201449 -0.00701727 -0.00013458  0.03127867 -0.02356948]
Criterion = [[ 0.00064195]]


Let's see if we can do better:

In [24]:
err_1 = err_vec(k_t, c_t, y_t, k_sim_t, c_sim_t, y_sim_t, False)
VCV2 = np.dot(err_1, err_1.T) / 100
print(VCV2)
W_hat2 = lin.pinv(VCV2)  # Use the pseudo-inverse calculated by SVD because VCV2 is ill-conditioned
print(W_hat2)

[[  1.11620263e-08  -2.12831457e-08   7.41378033e-08   1.42186702e-09
   -3.30460663e-07   2.49012702e-07]
 [ -2.12831457e-08   4.05815467e-08  -1.41361938e-07  -2.71113883e-09
    6.30104451e-07  -4.74803901e-07]
 [  7.41378033e-08  -1.41361938e-07   4.92420799e-07   9.44399296e-09
   -2.19490862e-06   1.65393400e-06]
 [  1.42186702e-09  -2.71113883e-09   9.44399296e-09   1.81123550e-10
   -4.20955037e-08   3.17203113e-08]
 [ -3.30460663e-07   6.30104451e-07  -2.19490862e-06  -4.20955037e-08
    9.78355071e-06  -7.37221905e-06]
 [  2.49012702e-07  -4.74803901e-07   1.65393400e-06   3.17203113e-08
   -7.37221905e-06   5.55520335e-06]]
[[  4.42458481e+01  -8.43655809e+01   2.93879436e+02   5.63622685e+00
   -1.30993352e+03   9.87076891e+02]
 [ -8.43655809e+01   1.60863709e+02  -5.60353354e+02  -1.07468514e+01
    2.49771011e+03  -1.88210463e+03]
 [  2.93879436e+02  -5.60353354e+02   1.95193734e+03   3.74356294e+01
   -8.70053440e+03   6.55613155e+03]
 [  5.63622685e+00  -1.07468514e+01 

In [26]:
params_init = np.array([alpha_SMM, rho_SMM, mu_SMM, sigma_SMM])
smm_args = (c_t, k_t, y_t, beta, unif_vals, W_hat2, S, T)
results2 = opt.minimize(criterion, params_init, args=(smm_args),
                          method='L-BFGS-B', bounds=((0.01, 1 - 0.01), (-1 + 0.01, 1 - 0.01),
                                                    (5, 14), (0.01, 1 + 0.01)))
alpha_SMM2, rho_SMM2, mu_SMM2, sigma_SMM2 = results2.x
print('alpha_SMM=', alpha_SMM, 'rho_SMM=', rho_SMM, 'mu_SMM=', mu_SMM,
      'sigma_SMM=', sigma_SMM)
print('alpha_SMM2=', alpha_SMM2, 'rho_SMM2=', rho_SMM2, 'mu_SMM2=', mu_SMM2,
      'sigma_SMM2=', sigma_SMM2)

alpha_SMM= 0.421003315084 rho_SMM= 0.99 mu_SMM= 9.91199426198 sigma_SMM= 0.0499435019757
alpha_SMM2= 0.406382490315 rho_SMM2= 0.988400177282 mu_SMM2= 10.0007535928 sigma_SMM2= 0.0782525250341


Estimates:

In [27]:
results2

      fun: array([[  1.61286010e-12]])
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -2.71437613e-05,  -1.59081312e-04,  -4.57716524e-05,
        -1.61298247e-04])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 55
      nit: 6
   status: 0
  success: True
        x: array([  0.40638249,   0.98840018,  10.00075359,   0.07825253])

In [28]:
(mean_ct_data, mean_kt_data, mean_ctyt_data, var_yt_data,
    corr_ctct_data, corr_ctkt_data) = data_moments(k_t, c_t, y_t)   
print('Data mean c_t =', mean_ct_data, ', Data mean k_t =', mean_kt_data,
     ', Data mean c_t/k_t =', mean_ctyt_data, ',\nData variance y_t =', var_yt_data,
     ', Data corr c_t c_t-1 =', corr_ctct_data, ',\nData corr c_t k_t =', corr_ctkt_data)
eps_t_SMM2 = sts.norm.ppf(unif_vals, loc=0, scale=sigma_SMM2)
z2_t, k_sim2_t, w_sim2_t, r_sim2_t, c_sim2_t, y_sim2_t = generator(alpha_SMM2, beta, rho_SMM2,
                                                                   mu_SMM2, sigma_SMM2,
                                                                   eps_t_SMM2, T, S)
(mean_ct_sim2, mean_kt_sim2, mean_ctyt_sim2, var_yt_sim2, corr_ctct_sim2,
corr_ctkt_sim2) = data_moments(c_sim2_t, k_sim2_t, y_sim2_t)
mean_ct_mod2 = np.mean(mean_ct_sim2)
mean_kt_mod2 = np.mean(mean_kt_sim2)
mean_ctyt_mod2 = np.mean(mean_ctyt_sim2)
var_yt_mod2 = np.mean(var_yt_sim2)
corr_ctct_mod2 = np.mean(corr_ctct_sim2)
corr_ctkt_mod2 = np.mean(corr_ctkt_sim2)
print('Model 2 mean c_t =', mean_ct_mod2, ', Model 2 mean k_t =', mean_kt_mod2,
     ', Model 2 mean c_t/k_t =', mean_ctyt_mod2, ',\nModel 2 variance y_t =', var_yt_mod2,
     ', Model 2 corr c_t c_t-1 =', corr_ctct_mod2, ',\nModel 2 corr c_t k_t =', corr_ctkt_mod2)
err_2 = err_vec(k_t, c_t, y_t, k_sim2_t, c_sim2_t, y_sim2_t, False).reshape(6,)
print('Error vector 2 =', err_2)
print('Criterion 2 =', results2.fun)
print('Model 1 mean c_t =', mean_ct_mod, ', Model 1 mean k_t =', mean_kt_mod,
     ', Model 1 mean c_t/k_t =', mean_ctyt_mod, ',\nModel 1 variance y_t =', var_yt_mod,
     ', Model 1 corr c_t c_t-1 =', corr_ctct_mod, ',\nModel 1 corr c_t k_t =', corr_ctkt_mod)
err_1 = err_vec(k_t, c_t, y_t, k_sim_t, c_sim_t, y_sim_t, False).reshape(6,)
print('Error vector 1 =', err_1)
print('Criterion 1 =', results1.fun)

Data mean c_t = 6643985.1383 , Data mean k_t = 9281790.48567 , Data mean c_t/k_t = 0.420745434314 ,
Data variance y_t = 2.83778250589e+13 , Data corr c_t c_t-1 = 0.941096816591 ,
Data corr c_t k_t = 0.940803053798
Model 2 mean c_t = 8331372.24548 , Model 2 mean k_t = 5609562.01039 , Model 2 mean c_t/k_t = 0.59456850544 ,
Model 2 variance y_t = 6.77787320837e+13 , Model 2 corr c_t c_t-1 = 0.926398220096 ,
Model 2 corr c_t k_t = 0.901374204429
Error vector 2 = [-0.15569317 -0.102396   -0.03639716  1.38843999  0.02348743 -0.04190978]
Criterion 2 = [[  1.61286010e-12]]
Model 1 mean c_t = 9300488.52455 , Model 1 mean k_t = 6636965.73538 , Model 1 mean c_t/k_t = 0.582207050049 ,
Model 1 variance y_t = 2.83740059102e+13 , Model 1 corr c_t c_t-1 = 0.93271224467 ,
Model 1 corr c_t k_t = 0.918628816083
Error vector 1 = [-0.0010565   0.00201449 -0.00701727 -0.00013458  0.03127867 -0.02356948]
Criterion 1 = [[ 0.00064195]]


Comparing Model 2 ($W_{two\;step}$) with Model 1 ($W = I$), it seems that we did better on $c_t$, but at the expense of the other moments. As usual, since we've changed the weighting matrix, the criterion function values are not comparable.

We can also check non-estimated models:

In [29]:
print('Data mean w_t =', np.mean(omega_t), ', Model 1 mean w_t =', np.mean(np.mean(w_sim_t, axis=0)),
     ', Model 2 mean w_t =', np.mean(np.mean(w_sim2_t, axis=0)))
print('Data var. c_t =', np.var(c_t), ', Model 1 var. c_t =', np.mean(np.var(c_sim_t, axis=0)),
     ', Model 2 var. c_t =', np.mean(np.var(c_sim2_t, axis=0)))
print('Data mean r_t =', np.mean(r_t), ', Model 1 mean r_t =', np.mean(np.mean(r_sim_t, axis=0)),
     ', Model 2 mean r_t =', np.mean(np.mean(r_sim2_t, axis=0)))

Data mean w_t = 9215060.73552 , Model 1 mean w_t = 9227733.1825 , Model 2 mean w_t = 8275582.67565
Data var. c_t = 9.68505769833e+12 , Model 1 var. c_t = 1.0121079561e+13 , Model 2 var. c_t = 2.55021400223e+13
Data mean r_t = 1.00930722271 , Model 1 mean r_t = 1.01072497137 , Model 2 mean r_t = 1.01024289292


Model 1 ($W = I$) seems to be better.